In [39]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import numpy as np

In [2]:
loan_data_backup = pd.read_csv(r'D:\Datasets\LendingClub\loan_data_2007_2014.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_2552\181822990.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_backup = pd.read_csv(r'D:\Datasets\LendingClub\loan_data_2007_2014.csv')


In [3]:
loan_data = loan_data_backup.copy()

In [4]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

### Preprocessing continuous variables

##### Employment length

In [5]:
loan_data['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [6]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')

C:\Users\User\AppData\Local\Temp\ipykernel_2552\2883447039.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years', '')


In [7]:
loan_data['empy_length_int'] = pd.to_numeric(loan_data['emp_length_int'])

In [8]:
loan_data['emp_length_int'].unique()

array(['10', '0', '1', '3', '8', '9', '4', '5', '6', '2', '7', nan],
      dtype=object)

##### Term

In [9]:
# Create 'term_int' with term in months
loan_data['term']

0          36 months
1          60 months
2          36 months
3          36 months
4          60 months
             ...    
466280     60 months
466281     60 months
466282     60 months
466283     36 months
466284     36 months
Name: term, Length: 466285, dtype: object

In [10]:
loan_data['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [11]:
loan_data['term_int'] = loan_data['term'].str.replace(' ', '')
loan_data['term_int'] = loan_data['term_int'].str.replace('months', '')
loan_data['term_int'] = pd.to_numeric(loan_data['term_int'])

In [12]:
loan_data['term_int']

0         36
1         60
2         36
3         36
4         60
          ..
466280    60
466281    60
466282    60
466283    36
466284    36
Name: term_int, Length: 466285, dtype: int64

##### Date of earliest credit line

In [13]:
loan_data['earliest_cr_line'][0]

'Jan-85'

In [14]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format='%b-%y')

In [15]:
loan_data['earliest_cr_line_date'][0]

Timestamp('1985-01-01 00:00:00')

In [16]:
loan_data['earliest_cr_line_date'].max()

Timestamp('2068-12-01 00:00:00')

In [17]:
# Fixing incorrectly imputed dates
loan_data['earliest_cr_line_date'] = [pd.to_datetime(str(e).replace('20', '19')) if e > pd.to_datetime('2017-12-01') else e for e in loan_data['earliest_cr_line_date']]

In [18]:
loan_data['earliest_cr_line_date'].max()

Timestamp('2011-11-01 00:00:00')

In [19]:
# Reference date for elapsed months since earliest credit line
pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']

0        12022 days
1         6819 days
2         5874 days
3         7974 days
4         8005 days
            ...    
466280    5358 days
466281    7488 days
466282    5844 days
466283    5417 days
466284    6513 days
Name: earliest_cr_line_date, Length: 466285, dtype: timedelta64[ns]

In [20]:
loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']) / np.timedelta64(1, 'M')))

In [21]:
loan_data['mths_since_earliest_cr_line']

0         395.0
1         224.0
2         193.0
3         262.0
4         263.0
          ...  
466280    176.0
466281    246.0
466282    192.0
466283    178.0
466284    214.0
Name: mths_since_earliest_cr_line, Length: 466285, dtype: float64

In [22]:
loan_data['mths_since_earliest_cr_line'].describe()

count    466256.000000
mean        242.491078
std          86.928944
min          73.000000
25%         184.000000
50%         226.000000
75%         285.000000
max         887.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [23]:
# the issue caused by having dates in the future yielding negative months since earliest credit line has been fixed
loan_data.loc[:, ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][loan_data['mths_since_earliest_cr_line'] < 0]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line


##### Loan issued date

In [24]:
loan_data['issue_d']

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_d, Length: 466285, dtype: object

In [25]:
# Create 'mths_since_issue_d' with months since issue date
loan_data['mths_since_issue_d'] = pd.to_datetime(loan_data['issue_d'], format='%b-%y')
loan_data['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - loan_data['mths_since_issue_d']) / np.timedelta64(1, 'M')))

In [26]:
loan_data['mths_since_issue_d']

0         72.0
1         72.0
2         72.0
3         72.0
4         72.0
          ... 
466280    47.0
466281    47.0
466282    47.0
466283    47.0
466284    47.0
Name: mths_since_issue_d, Length: 466285, dtype: float64

### Preprocessing discrete variables

In [27]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 81 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Unnamed: 0                   466285 non-null  int64         
 1   id                           466285 non-null  int64         
 2   member_id                    466285 non-null  int64         
 3   loan_amnt                    466285 non-null  int64         
 4   funded_amnt                  466285 non-null  int64         
 5   funded_amnt_inv              466285 non-null  float64       
 6   term                         466285 non-null  object        
 7   int_rate                     466285 non-null  float64       
 8   installment                  466285 non-null  float64       
 9   grade                        466285 non-null  object        
 10  sub_grade                    466285 non-null  object        
 11  emp_title                 

In [30]:
# k - 1 dummy variables to represent k categories
pd.get_dummies(loan_data['grade'], prefix='grade', prefix_sep=':')

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
466280,0,0,1,0,0,0,0
466281,0,0,0,1,0,0,0
466282,0,0,0,1,0,0,0
466283,1,0,0,0,0,0,0


In [29]:
loan_data['grade'].unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [31]:
loan_data_dummies = [pd.get_dummies(loan_data['grade'], prefix='grade', prefix_sep=':'),
                    pd.get_dummies(loan_data['sub_grade'], prefix='sub_grade', prefix_sep=':'),
                    pd.get_dummies(loan_data['home_ownership'], prefix='home_ownership', prefix_sep=':'),
                    pd.get_dummies(loan_data['verification_status'], prefix='verification_status', prefix_sep=':'),
                    pd.get_dummies(loan_data['loan_status'], prefix='loan_status', prefix_sep=':'),
                    pd.get_dummies(loan_data['purpose'], prefix='purpose', prefix_sep=':'),
                    pd.get_dummies(loan_data['addr_state'], prefix='addr_state', prefix_sep=':'),
                    pd.get_dummies(loan_data['initial_list_status'], prefix='initial_list_status', prefix_sep=':')]

In [32]:
loan_data_dummies = pd.concat(loan_data_dummies, axis=1)

In [33]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis=1)

In [34]:
loan_data.columns.values

array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'veri

### Check for missing values and clean

In [40]:
loan_data.isnull().sum()

Unnamed: 0                                                              0
id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_title                                                           27588
emp_length                                                          21008
home_ownership                        

In [ ]:
# variables to fill missing:
# annual_inc
# delinq_2yrs
# inq_last_6mths
# open_acc
# pub_rec
# total_acc
# acc_now_delinq
# total_rev_hi_lim
# emp_length_int
# mths_since_earliest_cr_line

In [41]:
# total_rev_hi_lim = total revolving limit
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace=True)
loan_data['total_rev_hi_lim'].isnull().sum()

0

In [42]:
loan_data['annual_inc'].fillna((loan_data['annual_inc'].dropna()).mean(), inplace=True)
loan_data['annual_inc'].isnull().sum()

0

In [50]:
for c in ['mths_since_earliest_cr_line', 'acc_now_delinq', 'total_acc', 'pub_rec', 'open_acc', 'inq_last_6mths', 'delinq_2yrs', 'emp_length_int']:
    loan_data[c].fillna(0, inplace=True)
    print(c + ' : ' + str(loan_data[c].isnull().sum()))

mths_since_earliest_cr_line : 0
acc_now_delinq : 0
total_acc : 0
pub_rec : 0
open_acc : 0
inq_last_6mths : 0
delinq_2yrs : 0
emp_length_int : 0


In [51]:
loan_data.to_csv(r'D:\Datasets\LendingClub\loan_data_processed.csv')